<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Build an AI Math Assistant with LangChain Tool Calling**

## Installing required libraries

The following required libraries are __not__ pre-installed in the Skills Network Labs environment. __You will need to run the following cell__ to install them:


In [ ]:
%pip install langchain==0.3.23 | tail -n 1
%pip install langchain-ibm==0.3.10 | tail -n 1
%pip install langchain-community==0.3.16 | tail -n 1
%pip install wikipedia==1.4.0 | tail -n 1
%pip install openai==1.77.0 | tail -n 1
%pip install langchain-openai==0.3.16 | tail -n 1

## Import the required libraries


In [ ]:
from langchain_ibm import ChatWatsonx
from langchain.agents import AgentType
import re

## Loading the LLM: Choosing the right language model

In this example, IBM’s `ChatWatsonx`will be used to load a language model (LLM) for interacting with tools. IBM’s models, like Granite 3.2 and Granite 3.3, are highly versatile and excel at advanced reasoning tasks.

That said, other providers offer LLMs with different strengths:

- **OpenAI (GPT-4/GPT-3.5)**: Best for versatility and advanced reasoning.
- **Facebook (Meta, LLaMA)**: Open-access, highly customizable for specialized use cases.
- **IBM watsonx Granite**: Ideal for enterprise applications with seamless integration.
- **Anthropic (Claude)**: Focused on safety, reliability, and ethical AI.
- **Cohere**: Affordable and efficient for lightweight, task-specific models.

---

For this project, you'll use `ChatWatsonx` because:
- It offers a simple API for quick setup.
- It supports advanced configurations like:
  - **`temperature`**: Adjusting randomness of responses.
  - **`max_tokens`**: Limiting the length of responses.
- IBM’s models are widely regarded as state-of-the-art for general-purpose reasoning and conversation.


In [ ]:
llm = ChatWatsonx(
    model_id="ibm/granite-3-2-8b-instruct",
    url="https://us-south.ml.cloud.ibm.com",
    project_id="skills-network",
)

Let's generate a simple response:


In [ ]:
response = llm.invoke("What is tool calling in langchain?")
print("\nResponse Content: ", response.content)

### Running Locally

If you are running this lab locally, you will need to configure your own API keys. This lab uses `ChatOpenAI` and `ChatWatsonx` modules from `langchain`. Both configurations are shown below with instructions. **Replace all instances** of both modules with the completed modules below throughout the lab. **DO NOT** run the cell below if you aren't running locally, it will causes errors.


## **Exercise: Create a power tool to calculate exponents**

#### **Objective**
In this exercise, you will create a custom tool that calculates the power of a number (e.g., \( x^y \)). You will then integrate this tool into an agent and test its functionality.

---

#### **Step 1: Create the power tool**

1. **Define the Tool Function**:
   - Create a Python function named `calculate_power` that takes a string as input. The string will contain two numbers: the base (\( x \)) and the exponent (\( y \)).
   - The function should extract the numbers, calculate \( x^y \), and return the result as a dictionary with the key `"result"`.


In [ ]:
def calculate_power(input_text: str) -> dict:
    """
    Calculates the power of a number (x^y).

    Parameters:
    - input_text (str): A string like "2, 3", "2 3", "5^2", or "2 to the power of 3".

    Returns:
    - dict: {"result": <calculated value>} or an error message.
    """
    # Try to extract expressions like "5^2"
    match = re.search(r"(\d+(?:\.\d+)?)\s*\^+\s*(\d+(?:\.\d+)?)", input_text)
    if match:
        base = float(match.group(1))
        exponent = float(match.group(2))
        return {"result": base ** exponent}

    # Try to extract expressions like "2 to the power of 3"
    match = re.search(r"(\d+(?:\.\d+)?)\s*(?:to\s+the\s+power\s+of)\s*(\d+(?:\.\d+)?)", input_text, re.IGNORECASE)
    if match:
        base = float(match.group(1))
        exponent = float(match.group(2))
        return {"result": base ** exponent}

    # Fallback: assume two numbers separated by space or comma
    try:
        numbers = [float(num) for num in input_text.replace(",", " ").split()]
        if len(numbers) != 2:
            return {"result": "Invalid input. Please provide exactly two numbers."}
        base, exponent = numbers
        return {"result": base ** exponent}
    except ValueError:
        return {"result": "Invalid input format. Provide input like '2 3', '2^3', or '2 to the power of 3'."}

2. **Create the tool object**:
   - Use the `Tool` class from LangChain to create a tool object for the `calculate_power` function.
   - Provide a name, description, and the function to the tool.


In [ ]:
power_tool = Tool(
   name="PowerTool",
   func=calculate_power,
   description="Calculates the power of a number (x^y). Input should be two numbers: base and exponent."
)

#### **Step 2: Create an agent with the power tool**

1. **Set up the agent**:
   - Use the `initialize_agent` function from LangChain to create an agent.
   - Include the `power_tool` in the list of tools provided to the agent.
   - Specify the agent type (e.g., `zero-shot-react-description`).


In [ ]:
# List of tools for the agent
tools = [power_tool]

# Create the agent
agent = initialize_agent(
   tools,
   llm,
   agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
   verbose=True,
    handle_parsing_errors=True
)

#### **Step 3: Test the agent**

1. **Test the Agent Using the `run` Function**:
   - Use the `run` function of the agent to test its ability to calculate powers.
   - Pass natural language queries to the agent and observe its responses.


In [ ]:
agent.run("Calculate 5 to the power of 2.")

Copyright © IBM Corporation. All rights reserved.
